In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from scipy import stats
import requests
from requests import get
from bs4 import BeautifulSoup
import xgboost as xgb
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import absolute
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from linearmodels import PooledOLS
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from sklearn.decomposition import PCA
import prince
import mca
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.linear_model import LassoCV
from numpy import arange

In [13]:
file = 'https://raw.githubusercontent.com/kalinakalina/DEDA_Cryptopunks/main/data/transaction.csv'
df_tr = pd.read_csv(file, header = None, index_col = 0)
df_tr.columns = ['type', 'from', 'to', 'date', 'punk_id', 'amount_in_eth', 'amount_in_dol']
df_tr = df_tr[ ['punk_id'] + [ col for col in df_tr.columns if col != 'punk_id' ] ]
df_tr.head()

,punk_id,type,from,to,date,amount_in_eth,amount_in_dol
0,,,,,,,
0,0,Bid Withdrawn,0x983ace,NaN,2021-05-10,250.0,1030000.0
1,0,Bid,0x983ace,NaN,2021-04-11,250.0,"537,615"
2,0,Bid Withdrawn,0xd7510a,NaN,2021-03-03,100.0,"160,973"
3,0,Bid,0xd7510a,NaN,2021-02-20,100.0,"188,897"
4,0,Bid Withdrawn,natealex,NaN,2020-09-16,69.0,"25,532"


In [14]:
def fix_df(x):
    x['punk_id'] = x['punk_id'].astype('category')
    x['type'] = x['type'].astype('category')
    x['from'] = x['from'].astype(str)
    x['to'] = x['to'].astype(str)
    x['date'] = pd.to_datetime(x['date'], format = '%Y.%m.%d')
    
    x['amount_in_eth'] = x['amount_in_eth'].astype(str)
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Y', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('Z', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('M', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('T', '')
    x['amount_in_eth'] = x['amount_in_eth'].str.replace('B', '')
    x['amount_in_eth'] = x['amount_in_eth'].replace(',', '', regex=True)
    x['amount_in_eth'] = x['amount_in_eth'].astype(float)

    x['amount_in_dol'] = x['amount_in_dol'].astype(str)
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Y', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('B', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('T', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('P', '')
    x['amount_in_dol'] = x['amount_in_dol'].str.replace('Z', '')
    x['amount_in_dol'] = x['amount_in_dol'].replace('<', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].replace(',', '', regex=True)
    x['amount_in_dol'] = x['amount_in_dol'].astype(float)
    
    x['amount_in_eth'] = x['amount_in_eth'].fillna(0)
    x['amount_in_dol'] = x['amount_in_dol'].fillna(0)

    return x

In [15]:
df_tr = fix_df(df_tr)

#Average Sale Price
sales = df_tr[df_tr['type'].str.contains("Sold")].reset_index(drop=True)
sales = sales.drop(['from', 'to', 'type', 'amount_in_eth'], axis=1)
sales = sales.groupby(['punk_id'])['amount_in_dol'].mean().reset_index()
sales = sales.fillna(0)
sales.head()

,punk_id,amount_in_dol
0,0,1176.000000
1,1,13855.666667
2,2,0.000000
3,3,0.000000
4,4,0.000000


In [16]:
file2 = 'https://raw.githubusercontent.com/kalinakalina/DEDA_Cryptopunks/main/data/accessories.csv'
df_acc = pd.read_csv(file2, header = None, sep='\n')
df_acc = df_acc[0].str.split(',', expand=True)
df_acc.drop(0, axis=1, inplace=True)

dict={1:'gender', 
      2:  'acc_1',
      3:'acc_2', 
      4: 'acc_3',
      5:'acc_4',
      6: 'acc_5',
      7:'acc_6',
      8:'acc_7'}
df_acc.rename(columns=dict,
          inplace=True)
  
df_acc.head()

,gender,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7
0,Female,Green Eye Shadow,Earring,Blonde Bob,None,None,None,None
1,Male,Smile,Mohawk,None,None,None,None,None
2,Female,Wild Hair,None,None,None,None,None,None
3,Male,Wild Hair,Nerd Glasses,Pipe,None,None,None,None
4,Male,Big Shades,Wild Hair,Earring,Goat,None,None,None


In [17]:
types = df_acc['gender']
types_dummies = pd.get_dummies(types)
types_dummies['punk_id'] = types_dummies.index
types_dummies = types_dummies[ ['punk_id'] + [ col for col in types_dummies.columns if col != 'punk_id' ] ]

In [18]:
file3 = 'https://raw.githubusercontent.com/kalinakalina/DEDA_Cryptopunks/main/data/accessories_transformed_full.csv'
acc = pd.read_csv(file3, index_col = 0)
acc['punk_id'] = acc.index
acc = acc[ ['punk_id'] + [ col for col in acc.columns if col != 'punk_id' ] ]
acc.head()

,punk_id,Beanie,Choker,Pilot Helmet,Tiara,Orange Side,Buck Teeth,Welding Goggles,Pigtails,Pink With Hat,...,Regular Shades,Horned Rim Glasses,Big Shades,Nerd Glasses,Black Lipstick,Mole,Purple Lipstick,Hot Lipstick,Cigarette,Earring
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [19]:
df = pd.merge(sales, types_dummies, on="punk_id", how='left')
df = pd.merge(df, acc, on="punk_id", how='left')

df = df.set_index(['punk_id'])
df = df.fillna(0)
df

,amount_in_dol,Alien,Ape,Female,Male,Zombie,Beanie,Choker,Pilot Helmet,Tiara,...,Regular Shades,Horned Rim Glasses,Big Shades,Nerd Glasses,Black Lipstick,Mole,Purple Lipstick,Hot Lipstick,Cigarette,Earring
punk_id,,,,,,,,,,,,,,,,,,,,,
0,1176.000000,0,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,13855.666667,0,0,0,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0,0,0,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0,0,0,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.000000,0,0,1,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0.000000,0,0,0,1,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9997,95416.000000,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Simple Linear Regression

In [20]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]
X = X.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [21]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print('Adjusted Rsquared: {}.'.format(1 - (1-regressor.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

Adjusted Rsquared: 0.17245715174791143.


In [22]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df = coeff_df.sort_values(by=['Coefficient'], ascending=False)
coeff_df

,Coefficient
Alien,2.189805e+17
Zombie,2.189805e+17
Ape,2.189805e+17
Female,2.189805e+17
Male,2.189805e+17
...,...
Headband,-1.942000e+03
Goat,-2.734000e+03
Knitted Cap,-3.456000e+03
Medical Mask,-6.554500e+03


In [23]:
y_pred = regressor.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 22819.017333333333
Mean Squared Error: 5022405005.189333
Root Mean Squared Error: 70868.92834796738


# Linear Regression with Lasso Regularization

In [24]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]
X = X.astype(int)

In [25]:
import warnings
warnings.filterwarnings('ignore')
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define model
model = LassoCV(alphas=arange(0, 1, 0.01), cv=cv, n_jobs=-1)
# fit model
model.fit(X, y)
# summarize chosen configuration
print('alpha: %f' % model.alpha_)

alpha: 0.840000


In [26]:
opt_model = Lasso(alpha=0.84)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)
opt_model.fit(X_train, y_train)
print('Adjusted Rsquared: {}.'.format(1 - (1-opt_model.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

Adjusted Rsquared: 0.12196003232050268.


In [27]:
coeff_df = pd.DataFrame(opt_model.coef_, X.columns, columns=['Coefficient'])
coeff_df = coeff_df.sort_values(by=['Coefficient'], ascending=False)
coeff_df

,Coefficient
Alien,573300.159005
Zombie,78677.322847
Ape,68579.642534
Orange Side,34167.047925
Pilot Helmet,31169.136987
...,...
Mohawk,-5682.013011
Police Cap,-6503.918895
Male,-6513.519322
Peak Spike,-6694.475288


In [28]:
y_pred = opt_model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 21522.69302146278
Mean Squared Error: 4527783666.869885
Root Mean Squared Error: 67288.80788712106


# Linear Regression with MCA

In [29]:
mca_acc = prince.MCA(
     n_components=8,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42
 )
mca_acc = mca_acc.fit(df_acc)
mca_acc.explained_inertia_

[0.017491918697282397,
 0.011619412955466613,
 0.011310967007782677,
 0.011115670453114113,
 0.010301098069866092,
 0.010226499546043153,
 0.00982573074770247,
 0.009706208772175538]

In [30]:
mca_acc = mca_acc.transform(df_acc)
mca_acc = pd.DataFrame(mca_acc)
mca_acc.index.name='punk_id'
mca_acc.head()

,0,1,2,3,4,5,6,7
punk_id,,,,,,,,
0,1.038836,0.013757,0.491787,-0.203250,-0.563574,-0.262089,-0.476637,-0.299301
1,-0.738621,0.155581,2.245116,-0.137410,1.143202,-0.091987,1.360325,1.928501
2,0.373272,0.144719,-0.207156,-1.046743,0.134286,-0.031461,-0.077573,-0.439775
3,-0.755384,0.075674,-0.109474,-0.807458,0.010501,-0.110404,0.163441,-0.754422
4,-0.210543,-0.175639,0.027385,0.388300,-0.442742,-0.005061,-0.254494,-0.047554


In [31]:
df = pd.merge(sales, mca_acc, on="punk_id", how='left')
df = df.set_index(['punk_id'])
df = df.fillna(0)
df

,amount_in_dol,0,1,2,3,4,5,6,7
punk_id,,,,,,,,,
0,1176.000000,1.038836,0.013757,0.491787,-0.203250,-0.563574,-0.262089,-0.476637,-0.299301
1,13855.666667,-0.738621,0.155581,2.245116,-0.137410,1.143202,-0.091987,1.360325,1.928501
2,0.000000,0.373272,0.144719,-0.207156,-1.046743,0.134286,-0.031461,-0.077573,-0.439775
3,0.000000,-0.755384,0.075674,-0.109474,-0.807458,0.010501,-0.110404,0.163441,-0.754422
4,0.000000,-0.210543,-0.175639,0.027385,0.388300,-0.442742,-0.005061,-0.254494,-0.047554
...,...,...,...,...,...,...,...,...,...
9995,0.000000,1.603686,0.041473,1.233879,-0.071900,-1.523943,-0.257354,-0.514234,0.473575
9996,0.000000,-0.681362,-0.236497,-0.243782,0.416840,-0.357151,-0.117103,-0.049904,-0.175537
9997,95416.000000,-0.868262,-0.404074,-0.140224,0.951132,-0.544511,0.116622,0.904179,1.276691


In [32]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [33]:
mca_reg = LinearRegression()
mca_reg.fit(X_train, y_train)
print('Adjusted Rsquared: {}.'.format(1 - (1-mca_reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

Adjusted Rsquared: 0.005430133218859212.


In [34]:
y_pred = mca_reg.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 21868.49592996465
Mean Squared Error: 5606689169.392453
Root Mean Squared Error: 74877.82828977115


# Linear Regression with Time Dummies

In [35]:
sales = df_tr[df_tr['type'].str.contains("Sold")].reset_index(drop=True)
sales = sales.drop(['from', 'to', 'type', 'amount_in_eth'], axis=1)
sales.head()

,punk_id,date,amount_in_dol
0,0,2018-11-30,2822.0
1,0,2017-07-07,386.0
2,0,2017-06-23,320.0
3,1,2020-11-30,36305.0
4,1,2019-04-06,5155.0


In [36]:
file4 = 'https://raw.githubusercontent.com/kalinakalina/DEDA_Cryptopunks/main/data/sales_time_dummie_encoded.csv'
time_dummies = pd.read_csv(file4, index_col = 0)
time_dummies = time_dummies.drop(['date', 'amount_in_dol'], axis = 1)
time_dummies = time_dummies.set_index('punk_id')
time_dummies.reset_index(level=0, inplace=True)
time_dummies.head()

,punk_id,monthyear_12018,monthyear_12019,monthyear_12020,monthyear_12021,monthyear_22018,monthyear_22019,monthyear_22020,monthyear_22021,monthyear_32018,...,monthyear_102019,monthyear_102020,monthyear_112017,monthyear_112018,monthyear_112019,monthyear_112020,monthyear_122017,monthyear_122018,monthyear_122019,monthyear_122020
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df = pd.merge(sales, types_dummies, on="punk_id", how='left')
df = pd.merge(df, acc, on="punk_id", how='left')

df = df.join(time_dummies, on=df.punk_id, how='left', lsuffix='', rsuffix='_')

df = df.set_index(['punk_id', 'date'])
df = df.fillna(0)
df

amount_in_dol  Alien  Ape  Female  Male  Zombie  Beanie  \
punk_id date                                                                  
0       2018-11-30         2822.0      0    0       1     0       0     0.0   
        2017-07-07          386.0      0    0       1     0       0     0.0   
        2017-06-23          320.0      0    0       1     0       0     0.0   
1       2020-11-30        36305.0      0    0       0     1       0     0.0   
        2019-04-06         5155.0      0    0       0     1       0     0.0   
...                           ...    ...  ...     ...   ...     ...     ...   
9990    2021-03-20        44981.0      0    0       0     1       0     0.0   
9997    2021-02-08       169770.0      0    0       0     0       1     0.0   
        2020-09-27        21062.0      0    0       0     0       1     0.0   
9998    2021-03-03        61170.0      0    0       1     0       0     0.0   
        2020-12-27         9499.0      0    0       1     0       0     0.0   

                    Choker  Pilot Helmet  Tiara  ...  monthyear_102019  \
punk_id date                                     ...                     
0       2018-11-30     0.0           0.0    0.0  ...                 0   
        2017-07-07     0.0           0.0    0.0  ...                 0   
        2017-06-23     0.0           0.0    0.0  ...                 0   
1       2020-11-30     0.0           0.0    0.0  ...                 0   
        2019-04-06     0.0           0.0    0.0  ...                 0   
...                    ...           ...    ...  ...               ...   
9990    2021-03-20     0.0           0.0    0.0  ...                 0   
9997    2021-02-08     0.0           0.0    0.0  ...                 0   
        2020-09-27     0.0           0.0    0.0  ...                 0   
9998    2021-03-03     0.0           0.0    0.0  ...                 0   
        2020-12-27     0.0           0.0    0.0  ...                 0   

                    monthyear_102020  monthyear_112017  monthyear_112018  \
punk_id date                                                               
0       2018-11-30                 0                 0                 1   
        2017-07-07                 0                 0                 1   
        2017-06-23                 0                 0                 1   
1       2020-11-30                 0                 0                 0   
        2019-04-06                 0                 0                 0   
...                              ...               ...               ...   
9990    2021-03-20                 0                 0                 0   
9997    2021-02-08                 0                 0                 0   
        2020-09-27                 0                 0                 0   
9998    2021-03-03                 0                 0                 0   
        2020-12-27                 0                 0                 0   

                    monthyear_112019  monthyear_112020  monthyear_122017  \
punk_id date                                                               
0       2018-11-30                 0                 0                 0   
        2017-07-07                 0                 0                 0   
        2017-06-23                 0                 0                 0   
1       2020-11-30                 0                 0                 0   
        2019-04-06                 0                 0                 0   
...                              ...               ...               ...   
9990    2021-03-20                 0                 0                 0   
9997    2021-02-08                 0                 0                 0   
        2020-09-27                 0                 0                 0   
9998    2021-03-03                 0                 0                 0   
        2020-12-27                 0                 0                 0   

                    monthyear_122018  monthyear_122

In [87]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]
X = X.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=888)

In [83]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
print('Adjusted Rsquared: {}.'.format(1 - (1-regressor.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

Adjusted Rsquared: 0.16861154040385729.


In [84]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df = coeff_df.sort_values(by=['Coefficient'], ascending=False)
coeff_df

,Coefficient
Alien,1.085693e+06
monthyear_42018,1.129006e+05
Orange Side,7.811073e+04
Pigtails,3.771274e+04
3D Glasses,3.732548e+04
...,...
monthyear_92018,-1.763981e+04
Ape,-1.556437e+05
Zombie,-2.316629e+05
Female,-3.465963e+05


In [85]:
y_pred = regressor.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 29378.239067535196
Mean Squared Error: 12821449284.684755
Root Mean Squared Error: 113231.83865276036


# Panel Data Models with Time Dummies

In [38]:
y = df['amount_in_dol']
y = y.round(decimals=-2)

X = df.iloc[:,1:]
X = X.astype(int)

In [39]:
exog = sm.tools.tools.add_constant(X)
exog = exog.astype(int)
endog = y

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.3, random_state=888)

model_re = RandomEffects(y_train, X_train, check_rank=False) 
re_res = model_re.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:          amount_in_dol   R-squared:                        0.1810
Estimator:              RandomEffects   R-squared (Between):              0.1722
No. Observations:                9471   R-squared (Within):               0.0000
Date:                Wed, Jun 30 2021   R-squared (Overall):              0.1810
Time:                        23:08:41   Log-likelihood                -1.211e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      14.621
Entities:                        5591   P-value                           0.0000
Avg Obs:                       1.6940   Distribution:                F(141,9329)
Min Obs:                       0.0000                                           
Max Obs:                       13.000   F-statistic (robust):             14.664
                            

In [40]:
y_pred = re_res.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 29378.239067534963
Mean Squared Error: 12821449284.684647
Root Mean Squared Error: 113231.83865275988
